In [1]:
import sys
from pathlib import Path

# Add src directory to Python path
project_root = Path.cwd().parent  
src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))
    print(f"Added {src_path} to Python path")


Added /Users/cooper/Desktop/hydro-forecasting/src to Python path


In [2]:
from hydro_forecasting.data.lazy_datamodule import HydroLazyDataModule
from hydro_forecasting.preprocessing.grouped import GroupedPipeline
from sklearn.pipeline import Pipeline
from hydro_forecasting.preprocessing.standard_scale import StandardScaleTransformer
from hydro_forecasting.data.caravanify_parquet import (
    CaravanifyParquet,
    CaravanifyParquetConfig,
)

from hydro_forecasting.models.tide import LitTiDE, TiDEConfig
from hydro_forecasting.model_evaluation.evaluators import TSForecastEvaluator


---

In [3]:
config_ca = CaravanifyParquetConfig(
    attributes_dir="/Users/cooper/Desktop/CaravanifyParquet/CA/post_processed/attributes",
    timeseries_dir="/Users/cooper/Desktop/CaravanifyParquet/CA/post_processed/timeseries/csv",
    shapefile_dir="/Users/cooper/Desktop/CAMELS-CH/data/CARAVANIFY/CA/post_processed/shapefiles",
    gauge_id_prefix="CA",
    use_hydroatlas_attributes=True,
    use_caravan_attributes=True,
    use_other_attributes=True,
)

caravan_ca = CaravanifyParquet(config_ca)
basin_ids = caravan_ca.get_all_gauge_ids()[:10]

# basin_ids = [bid for bid in basin_ids if bid != "CA_15030"]

In [4]:
config_us = CaravanifyParquetConfig(
    attributes_dir="/Users/cooper/Desktop/CaravanifyParquet/USA/post_processed/attributes",
    timeseries_dir="/Users/cooper/Desktop/CaravanifyParquet/USA/post_processed/timeseries/csv",
    shapefile_dir="/Users/cooper/Desktop/CAMELS-CH/data/CARAVANIFY/USA/post_processed/shapefiles",
    gauge_id_prefix="USA",
    use_hydroatlas_attributes=True,
    use_caravan_attributes=True,
    use_other_attributes=True,
)

caravan_us = CaravanifyParquet(config_us)
basin_ids += caravan_us.get_all_gauge_ids()[:10]

In [5]:
basin_ids

['CA_15013',
 'CA_15016',
 'CA_15020',
 'CA_15022',
 'CA_15025',
 'CA_15030',
 'CA_15034',
 'CA_15039',
 'CA_15040',
 'CA_15044',
 'USA_01013500',
 'USA_01022500',
 'USA_01030500',
 'USA_01031500',
 'USA_01047000',
 'USA_01052500',
 'USA_01054200',
 'USA_01055000',
 'USA_01057000',
 'USA_01073000']

In [6]:
forcing_features = [
    "snow_depth_water_equivalent_mean",
    "surface_net_solar_radiation_mean",
    "surface_net_thermal_radiation_mean",
    "potential_evaporation_sum_ERA5_LAND",
    "potential_evaporation_sum_FAO_PENMAN_MONTEITH",
    "temperature_2m_mean",
    "temperature_2m_min",
    "temperature_2m_max",
    "total_precipitation_sum",
]

static_features = [
    # "gauge_id",
    "p_mean",
    "area",
    "ele_mt_sav",
    "high_prec_dur",
    "frac_snow",
    "high_prec_freq",
    "slp_dg_sav",
    "cly_pc_sav",
    "aridity_ERA5_LAND",
    "aridity_FAO_PM",
]

In [7]:
feature_pipeline = GroupedPipeline(
    Pipeline([("scaler", StandardScaleTransformer())]),
    columns=forcing_features,
    group_identifier="gauge_id",
)

target_pipeline = GroupedPipeline(
    Pipeline([("scaler", StandardScaleTransformer())]),
    columns=["streamflow"],
    group_identifier="gauge_id",
)

static_pipeline = Pipeline([("scaler", StandardScaleTransformer())])

preprocessing_config = {
    "features": {"pipeline": feature_pipeline},
    "target": {"pipeline": target_pipeline},
    "static_features": {"pipeline": static_pipeline, "columns": static_features},
}

In [8]:
region_time_series_base_dirs = {
    "CA": "/Users/cooper/Desktop/CaravanifyParquet/CA/post_processed/timeseries/csv/CA",
    "USA": "/Users/cooper/Desktop/CaravanifyParquet/USA/post_processed/timeseries/csv/USA",
}

region_static_attributes_base_dirs = {
    "CA": "/Users/cooper/Desktop/CaravanifyParquet/CA/post_processed/attributes/CA",
    "USA": "/Users/cooper/Desktop/CaravanifyParquet/USA/post_processed/attributes/USA",
}

datamodule = HydroLazyDataModule(
    region_time_series_base_dirs=region_time_series_base_dirs,
    region_static_attributes_base_dirs=region_static_attributes_base_dirs,
    path_to_preprocessing_output_directory="/Users/cooper/Desktop/hydro-forecasting/tests/yolo_6",
    group_identifier="gauge_id",
    batch_size=2048,
    input_length=70,
    output_length=10,
    forcing_features=forcing_features,
    static_features=static_features,
    target="streamflow",
    preprocessing_configs=preprocessing_config,
    num_workers=4,
    min_train_years=5,
    train_prop=0.5,
    val_prop=0.25,
    test_prop=0.25,
    max_imputation_gap_size=5,
    list_of_gauge_ids_to_process=basin_ids,
    is_autoregressive=True,
    files_per_batch=20,
)

## Let's try training a model

In [9]:
input_length = datamodule.input_length
output_length = datamodule.output_length

config = TiDEConfig(
    input_len=input_length,
    output_len=output_length,
    input_size=len(forcing_features),
    future_input_size=len(forcing_features),
    static_size=len(static_features),
    num_encoder_layers=2,
    num_decoder_layers=2,
    decoder_output_size=16,
    hidden_size=16,
    temporal_decoder_hidden_size=16,
    past_feature_projection_size=4,
    future_forcing_projection_size=4,
    use_layer_norm=True,
    dropout=0.1,
    learning_rate=1e-3,
)

# Instantiate the Lightning module.
model = LitTiDE(config)

In [10]:
import pytorch_lightning as pl

trainer = pl.Trainer(
    max_epochs=1,
    accelerator="gpu",
    devices=1,
)

# Train the model
trainer.fit(model, datamodule)

You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/cooper/Desktop/hydro-forecasting/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default



================ STARTING PREPROCESSING PIPELINE ================

================ LOADING STATIC ATTRIBUTES ================
INFO: Attempting to load static attributes for 20 gauge IDs
INFO: Loaded static attributes for 20 unique basins from 6 files.
INFO: Successfully loaded static attributes for 20 basins.

================ FITTING PREPROCESSING PIPELINES ================
INFO: Loading time series data for pipeline fitting...
INFO: Loaded time series data for 20 basins
INFO: Split time series data into train (96211), val (48099), test (48121)
INFO: Fitted 3 pipelines

================ PROCESSING BASIN TIME SERIES ================


Processing basins: 100%|██████████| 20/20 [00:01<00:00, 18.10it/s]



================ PROCESSING STATIC ATTRIBUTES ================
INFO: Processing static attributes...
SUCCESS: Saved transformed static attributes for 19 basins to /Users/cooper/Desktop/hydro-forecasting/tests/yolo_6/processed_static_data.parquet

================ PROCESSING SUMMARY ================
SUCCESS: Completed processing 19 of 20 basins



  | Name          | Type      | Params | Mode 
----------------------------------------------------
0 | mse_criterion | MSELoss   | 0      | train
1 | model         | TiDEModel | 22.5 K | train
----------------------------------------------------
22.5 K    Trainable params
0         Non-trainable params
22.5 K    Total params
0.090     Total estimated model params size (MB)
61        Modules in train mode
0         Modules in eval mode


INFO: Created training dataset with 92772 samples
INFO: Created validation dataset with 47502 samples


/Users/cooper/Desktop/hydro-forecasting/.venv/lib/python3.12/site-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (46) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 0: 100%|██████████| 46/46 [00:08<00:00,  5.22it/s, v_num=9]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 46/46 [00:08<00:00,  5.19it/s, v_num=9]


In [11]:
models_and_datamodules = {
    "TiDE": (model, datamodule),
}

evaluator = TSForecastEvaluator(
    horizons=list(range(1, output_length + 1)),
    models_and_datamodules=models_and_datamodules,
    trainer_kwargs={
        "accelerator": "gpu",
        "devices": 1,
    },
)

In [12]:
results = evaluator.test_models()

You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Testing TiDE...

================ STARTING PREPROCESSING PIPELINE ================

================ LOADING STATIC ATTRIBUTES ================
INFO: Attempting to load static attributes for 19 gauge IDs
INFO: Loaded static attributes for 19 unique basins from 6 files.
INFO: Successfully loaded static attributes for 19 basins.

================ FITTING PREPROCESSING PIPELINES ================
INFO: Loading time series data for pipeline fitting...
INFO: Loaded time series data for 19 basins
INFO: Split time series data into train (95566), val (47777), test (47797)
INFO: Fitted 3 pipelines

================ PROCESSING BASIN TIME SERIES ================


Processing basins: 100%|██████████| 19/19 [00:01<00:00, 16.70it/s]



================ PROCESSING STATIC ATTRIBUTES ================
INFO: Processing static attributes...
SUCCESS: Saved transformed static attributes for 19 basins to /Users/cooper/Desktop/hydro-forecasting/tests/yolo_6/processed_static_data.parquet

================ PROCESSING SUMMARY ================
SUCCESS: Completed processing 19 of 19 basins
INFO: Created test dataset with 47212 samples
Testing DataLoader 0: 100%|██████████| 24/24 [00:02<00:00, 11.54it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss                   nan
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Evaluating results with shape: preds=(47212, 10), obs=(47212, 10), basin_ids=(47212,)


TypeError: can't convert mps:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [ ]:
results